In [ ]:
import tensorflow



ozellik_cıkaran_model = tensorflow.keras.applications.VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(224, 224, 3)   
                  )

ozellik_cıkaran_model.summary()

ozellik_cıkaran_model.trainable = True
set_trainable = False
for layer in ozellik_cıkaran_model.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True     #Kullandığım VGG16 modelinin block5'ten sonraki evrişim katmanlarını kullanıyorum.
    else:
        layer.trainable = False

model = tensorflow.keras.models.Sequential()  #boş bir model oluşturdum.

model.add(ozellik_cıkaran_model)


model.add(tensorflow.keras.layers.Flatten()) #vektör haline getirdim.

model.add(tensorflow.keras.layers.Dense(256, activation='relu')) #Öğrenme yeteneğini artırmak için 256 nöron içeren katman
model.add(tensorflow.keras.layers.Dense(102, activation='softmax'))

model.compile(loss='binary_crossentropy',
              optimizer=tensorflow.keras.optimizers.RMSprop(lr=1e-5),
              metrics=['acc'])

model.summary()

egitim_yolu = r'C:\Users\pence\Desktop\bitkitanıma\veriseti\egitim'
gecerleme_yolu = r'C:\Users\pence\Desktop\bitkitanıma\veriseti\gecerleme'
test_yolu = r'C:\Users\pence\Desktop\bitkitanıma\veriseti\test'

train_datagen = tensorflow.keras.preprocessing.image.ImageDataGenerator(
      rescale=1./255, #piksel değerlerini 0-255'den 0-1 arasına getirdim.
      rotation_range=40, 
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,              # Verilen veriyi gerçek hayattaki görüntülere adapte etme ,veri artırımı işlemleri
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest'
      )

train_generator = train_datagen.flow_from_directory(
        egitim_yolu,
        target_size=(224, 224),
        batch_size=20, #Her seferde okunan veri sayısı
        )

validation_datagen = tensorflow.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255
        )

validation_generator = validation_datagen.flow_from_directory(
        gecerleme_yolu,
        target_size=(224, 224),
        batch_size=20,
        )

history = model.fit(
      train_generator,
      steps_per_epoch=100,
      epochs=75,
      validation_data=validation_generator,
      validation_steps=5)

model.save('bitkimodel.h5')

test_datagen = tensorflow.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255
        )

test_generator = test_datagen.flow_from_directory(
        test_yolu,
        target_size=(224, 224),
        batch_size=20,
        )

test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)
print('test acc:', test_acc)


In [ ]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QLabel, QVBoxLayout, QHBoxLayout, QPushButton, QFileDialog, QTextEdit, QMessageBox
from PyQt5.QtGui import QPixmap, QPainter, QFont, QColor
from PyQt5.QtCore import Qt
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions
import numpy as np
import json

class ImageApp(QWidget):
    def __init__(self):
        super().__init__()

        self.model = tf.keras.models.load_model('bitkimodel.h5')
        
        with open('bitkibilgisi.json', 'r', encoding='utf-8') as file:
            self.bitki_bilgileri = json.load(file)

        self.bitki_siniflari = {
            0: 'Artabotrys',
            1: 'Asarum',
            2: 'Asystasia',
            3: 'Bambuluotu', 
            4: 'Barleria',
            5: 'Beaumontia',
            6: 'Boeica',
            7: 'Brunfelsia',
            8: 'Caesalpinia',
            9: 'Calliandra',
            10: 'Dichroa',
            11: 'Dillenia',
            12: 'Duhaldea',
            13: 'Erythrina',
            14: 'Geophila',
            15: 'Gomphrena',
            16: 'Hewittia',
            17: 'Hoya',
            18: 'Hylocereus',
            19: 'Sarıkantaron',
            20: 'Abelia',
            21: 'Acıçiğdem',
            22: 'Adina',
            23: 'Agacmeneksesi',
            24: 'Akasya',
            25: 'Allamanda',
            26: 'Arjuna',
            27: 'Aslanagzı',
            28: 'Aster',
            29: 'Atatürkçiçeği',
            30: 'Atlasçiçeği',
            31: 'Ayciceği',
            32: 'Bahçegülü',
            33: 'Bakopa',
            34: 'Begonvil',
            35: 'Bergamot',
            36: 'Bodrumlalesi',
            37: 'Borucicegi',
            38: 'Cancicegi',
            39: 'Carkıfelek',
            40: 'Cennetkusu',
            41: 'Cınar',
            42: 'Defne',
            43: 'Devedikeni',
            44: 'Dönbaba',
            45: 'Ebegümeci',
            46: 'Ekinezya',
            47: 'Eçeverya',
            48: 'Farekulagı',
            49: 'Feslegen',
            50: 'Flamingocicegi',
            51: 'Gardenya',
            52: 'Gazanya',
            53: 'Gelinciceği',
            54: 'Gentiyan',
            55: 'Glayöl',
            56: 'Gül',
            57: 'Gündüzsefası',
            58: 'Güzelhatmi',
            59: 'Hasekiküpesi',
            60: 'Hercaimenekse',
            61: 'Hintmabetagacı',
            62: 'Hoşmeyve',
            63: 'Hüsnüyusuf',
            64: 'İpekotu',
            65: 'Japongülü',
            66: 'Jerbera',
            67: 'Kadifeciceği',
            68: 'Kaliforniyagelincigi',
            69: 'Kasımpatı',
            70: 'Kazgagası',
            71: 'Kelebekcalısı',
            72: 'Küfeya',
            73: 'Lale',
            74: 'Latincicegi',
            75: 'Loğusaotu',
            76: 'Manolya',
            77: 'Meksikaaycicegi',
            78: 'Mutafanlale',
            79: 'Nilüfer',
            80: 'Orkide',
            81: 'Ortanca',
            82: 'Papatya',
            83: 'Petunya',
            84: 'Sakayık',
            85: 'Saraypatı',
            86: 'Sardunya',
            87: 'Siklamen',
            88: 'Süguveyiotu',
            89: 'Süsen',
            90: 'Sütleğen',
            91: 'Vapurdumanı',
            92: 'Yabanienginar',
            93: 'Yabanikaranfil',
            94: 'Yalancıkecisakalı',
            95: 'Yıldızcicegi',
            96: 'Zambak',
            97: 'Çeşmepapatyası',
            98: 'Çuhaçiçeği',
            99: 'Çölgülü',
            100: 'Çöpleme',
            101: 'Öksürükotu'
        }


        

        self.initUI()

    def initUI(self):
        self.image_label = QLabel('Resim Gösterilecek Yer')
        self.image_label.setAlignment(Qt.AlignCenter)

        self.bitki_adi_label = QLabel('Bitki Adı: Bilinmiyor')
        self.bitki_adi_label.setFont(QFont("Arial", 12, QFont.Bold))
        self.keşif_yili_label = QLabel('Bitki Olasılığı: Bilinmiyor')
        self.keşif_yili_label.setFont(QFont("Arial", 10))

        self.file_button = QPushButton('Resim Seç')
        self.file_button.clicked.connect(self.showDialog)
        self.file_button.setStyleSheet("background-color: #4CAF50; color: white;")

        self.analiz_button = QPushButton('Analiz Et')
        self.analiz_button.clicked.connect(self.analyze)
        self.analiz_button.setStyleSheet("background-color: #008CBA; color: white;")

        self.bitki_bilgisi_text = QTextEdit()
        self.bitki_bilgisi_text.setPlaceholderText('Bitki bilgisi burada görüntülenecek.')
        self.bitki_bilgisi_text.setFont(QFont("Arial", 10))

        left_layout = QVBoxLayout()
        left_layout.addWidget(self.bitki_adi_label)
        left_layout.addWidget(self.keşif_yili_label)

        center_layout = QVBoxLayout()
        center_layout.addWidget(self.file_button)
        center_layout.addWidget(self.analiz_button)

        right_layout = QVBoxLayout()
        right_layout.addWidget(self.bitki_bilgisi_text)

        main_layout = QHBoxLayout()
        main_layout.addWidget(self.image_label)
        main_layout.addLayout(left_layout)
        main_layout.addLayout(center_layout)
        main_layout.addLayout(right_layout)

        self.setLayout(main_layout)

        self.setGeometry(300, 300, 800, 500)
        self.setWindowTitle('Bitki Analiz Uygulaması')
    
    def paintEvent(self, event):
        painter = QPainter(self)
        painter.setBrush(QColor("#C2E1C2")) 
        painter.drawRect(self.rect())

    def showDialog(self):
        fname, _ = QFileDialog.getOpenFileName(self, 'Resim Seç', '', 'Resim Dosyaları (*.png; *.jpg; *.jpeg)')

        if fname:
            pixmap = QPixmap(fname)
            self.image_label.setPixmap(pixmap)
            self.image_label.setScaledContents(True)
            self.image_label.setMinimumSize(1, 1)
            self.image_label.setMaximumSize(self.image_label.sizeHint())
        else:
            self.image_label.clear()

    def analyze(self):
        if self.image_label.pixmap() is None:
            self.showAlert('Lütfen resim seçiniz.')
            return

        img_path = 'temp_image.jpg'  
        self.image_label.pixmap().toImage().save(img_path)
        img = image.load_img(img_path, target_size=(224, 224))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)

        # Tahmin yap
        predictions = self.model.predict(img_array)

        # En yüksek olasılıklı sınıfın indeksini bul
        top_prediction_index = np.argmax(predictions[0])

        # Sınıf indeksini bitki ismine çevir
        if top_prediction_index in self.bitki_siniflari:
            bitki_adi = self.bitki_siniflari[top_prediction_index]
        else:
            bitki_adi = "Bilinmeyen"
            
        if bitki_adi in self.bitki_bilgileri:
            bitki_bilgisi = self.bitki_bilgileri[bitki_adi]
        else:
            bitki_bilgisi = {'bilgi': 'Bilgi bulunamadı.'}
    

        bitki_olasilik = predictions[0][top_prediction_index]

        # İlk tahminin sonucunu ekrana yaz
        self.bitki_adi_label.setText(f'Bitki Adı: {bitki_adi}')
        self.keşif_yili_label.setText(f'Bitki Olasılığı: {bitki_olasilik:.2%}')

        bitki_bilgisi_metin = f"""
        
        Bilgiler:
        {bitki_bilgisi['bilgi']}
        
        
        """
        self.bitki_bilgisi_text.setPlainText(bitki_bilgisi_metin)

    def showAlert(self, message):
        msg_box = QMessageBox()
        msg_box.setIcon(QMessageBox.Warning)
        msg_box.setText(message)
        msg_box.setWindowTitle('Uyarı')
        msg_box.exec_()

if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = ImageApp()
    ex.show()
    sys.exit(app.exec_())


1/1 [==============================] - 0s 156ms/step


In [ ]:
class_indices_map = train_generator.class_indices
print(class_indices_map)
